### TF IDF preporccesing


In [ ]:
import os
import sys

import pandas as pd
from ast import literal_eval
# from scipy.stats import friedmanchisquare
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

: 

In [2]:
current_dir = os.getcwd()

In [3]:
sys.path.append(os.path.join(current_dir, '..', 'src'))

In [4]:
import preprocessing_helper, colab_utils

[nltk_data] Downloading package punkt to /Users/jfraeys/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/jfraeys/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfraeys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jfraeys/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jfraeys/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# Define file paths
project_filenames = {
    "job_desc_filename": "data/interim/coop_postings_programs_joined.csv",
    "foundal_skills_filename": "data/interim/soft_skills.csv",
    "post_proc_job_post": "data/interim/post_processing_job_timeline.csv",
    "post_proc_found_skills": "data/interim/post_processing_soft_skills.csv",
    "replace_skills_job_post": "data/interim/replace_skills_job_timeline.csv",
}

In [6]:
try:
    env_info = colab_utils.setup_environment(
        filenames=project_filenames,
        colab_project_path=current_dir
    )
    if env_info["in_colab"]:
        print("Google Colab detected.")
    else:
        print("Not running in Google Colab.")
    print(f"Job Description File: {env_info['job_desc_filename']}")
    print(f"Foundational Skills File: {env_info['foundal_skills_filename']}")
except Exception as e:
    print(f"Error setting up environment: {e}")

Not running in Google Colab.
Job Description File: /Users/jfraeys/Library/CloudStorage/GoogleDrive-jfraeys@gmail.com/My Drive/School/UoGuelph/MSc/Research/soft_skills_job_desc/jfraeysd/data/interim/coop_postings_programs_joined.csv
Foundational Skills File: /Users/jfraeys/Library/CloudStorage/GoogleDrive-jfraeys@gmail.com/My Drive/School/UoGuelph/MSc/Research/soft_skills_job_desc/jfraeysd/data/interim/soft_skills.csv


In [ ]:
del current_dir

# Pre-processing job description

In [7]:
# Read job description data
df_coop = pd.read_csv(env_info['job_desc_filename'])

In [8]:
# df_coop["job_description_processed"] = preprocessing_helper.preprocess_text(df_coop["job_description"])

In [9]:
# df_coop.to_csv(post_proc_job_post, index=False)

In [10]:
df_coop = pd.read_csv(env_info['post_proc_job_post'])

# Pre-processing foundational skills

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df_soft_skills = pd.read_csv(env_info['foundal_skills_filename'])

In [ ]:
def process_labels(df: pd.DataFrame):
    # Import soft skills data
    # df_soft_skills = pd.read_csv(found_skills_filename))

    # Import soft skills data
    # df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill'].apply(lambda x: x.split())
    # df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels'].apply(lambda x: x.split(", "))

    # df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill_lemm'].apply(preprocessing_helper.preprocess_labels)
    # df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels_lemm'].apply(preprocessing_helper.preprocess_labels)

    # df_soft_skills['soft_skill_lemm'] = df_soft_skills['soft_skill_lemm'].apply(' '.join)
    # df_soft_skills['alt_labels_lemm'] = df_soft_skills['alt_labels_lemm'].apply(lambda x: list(set(x)))

    # df_soft_skills.to_csv(post_proc_found_skills, index=False)

In [ ]:
df_soft_skills = preprocessing_helper.load_or_process_feather(env_info['post_proc_found_skills'])

In [18]:
# Apply lemmatization to each list of 'alt_labels', creating 'alt_labels_lemm' column
df_soft_skills['alt_labels_lemm']=df_soft_skills['alt_labels_lemm'].apply(literal_eval)

# Replacing Alternate Labels with Foundational Skills

In [19]:
word_mapping = dict(zip(df_soft_skills['soft_skill_lemm'], df_soft_skills['alt_labels_lemm']))

In [20]:
# df_coop['job_description_replaced_skills']=df_coop['job_description_processed'].progress_apply(lambda x: preprocessing_helper.replace_keywords(x, word_mapping))

In [21]:
# df_coop.to_csv(env_info['replace_skills_job_post'], index=False)

In [22]:
df_coop = pd.read_csv(env_info['replace_skills_job_post'])

In [23]:
df_coop["year"] = pd.to_datetime(
    df_coop["date_goes_live"], format="%Y-%m-%d %H:%M:%S"
).dt.year

In [24]:
df_coop.groupby('year').size()

year
2012     424
2013    1208
2014    1345
2015    1538
2016    1635
2017    1785
2018    2045
2019    2223
2020    1675
2021    1961
2022    2097
2023    1073
dtype: int64

## Analysis

In [25]:
df_coop_processed = df_coop.copy()

In [26]:
# Assuming you have a DataFrame df_coop with job postings and a datetime column
tfidf_vectorizer = TfidfVectorizer(
    # max_df=0.65,
    # min_df=7,
    ngram_range=(1, 2),
    norm="l2",
)

tfidf_joined = []
unique_datetimes = df_coop_processed["year"].unique()

for dt_year in tqdm(unique_datetimes, desc="Running TF-IDF analysis by year"):
    df_year = df_coop_processed[df_coop_processed["year"] == dt_year]
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_year["job_description_replaced_skills"])
    df_tfidf = pd.DataFrame(
        tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
    )
    keyword_weights = df_tfidf.sum().reset_index()
    keyword_weights.columns = ["keyword", "tf_idf_weight"]
    sorted_keyword_weights = keyword_weights.sort_values(
        by="tf_idf_weight", ascending=False
    )
    tfidf_joined.append((dt_year, sorted_keyword_weights))

Running TF-IDF analysis by year:   0%|          | 0/12 [00:00<?, ?it/s]

In [27]:
# Assuming df_soft_skills contains the list of all unique foundational skills
all_foundational_skills = df_soft_skills["soft_skill_lemm"].tolist()

# Create a DataFrame with all soft skills
all_skills_df = pd.DataFrame({"keyword": all_foundational_skills})

# Create an empty DataFrame to store the yearly results
yearly_foundational_skills_df = pd.DataFrame()

# Iterate over yearly TF-IDF results
for year, yearly_tfidf_df in tfidf_joined:
    # Merge with all_skills_df to include all soft skills
    yearly_skills_df = all_skills_df.merge(yearly_tfidf_df, on="keyword", how="left")
    
    # Add the "year" column
    yearly_skills_df["year"] = year
    
    # Fill NaN values for zero-value columns with float 0
    zero_value_cols = yearly_tfidf_df.columns[1:]
    yearly_skills_df[zero_value_cols] = yearly_skills_df[zero_value_cols].fillna(0.00)
    
    # Append to the overall DataFrame
    yearly_foundational_skills_df = pd.concat([yearly_foundational_skills_df, yearly_skills_df], ignore_index=True)

In [28]:
pivot_table = yearly_foundational_skills_df.pivot_table(index='year', columns='keyword', values='tf_idf_weight', fill_value=0.0)

In [29]:
pivot_table

keyword,accountability,active listen,adaptive,analytical,argumentation,coach,commitment,communication,conceptual,conflict management,...,oral communication,passion,persuasion,presentation,problem solve,self confidence,self organize,social skill,speak,strategic think
year,,,,,,,,,,,,,,,,,,,,,
2012,5.288147,7.015836,0.573389,2.953944,0.961566,0.304964,2.738525,3.114192,1.687327,0.000000,...,0.561902,2.122319,0.000000,2.122252,2.916895,0.659578,0.000000,0.065533,0.508012,2.013902
2013,14.495105,18.871601,2.271504,8.417204,3.602829,0.926394,7.166819,7.066911,4.839000,0.093575,...,1.714082,4.962024,0.041401,6.932954,6.290471,1.692334,0.000000,0.260520,1.399236,5.866607
2014,17.301820,19.609159,2.787596,8.839863,3.746670,0.618730,6.591467,8.649410,5.088829,0.340164,...,2.026834,4.512568,0.055788,6.371816,6.911003,2.023198,0.000000,0.183462,1.437729,6.684688
2015,18.941407,23.210459,3.518101,10.073015,3.833663,1.005492,7.846545,10.601702,5.607145,0.146371,...,1.916553,5.780038,0.000000,7.430137,8.428106,1.744296,0.000000,0.053724,1.218875,7.028910
2016,19.969392,27.303306,4.280230,11.011923,4.132873,2.034285,8.415771,10.334202,6.221781,0.140398,...,2.297335,8.623162,0.080923,7.023985,9.093815,1.921759,0.000000,0.078597,1.326191,8.220760
2017,20.918112,27.051080,4.593396,12.210576,5.369378,2.889582,9.780092,12.769102,6.966879,0.235837,...,2.574042,9.636338,0.145285,8.456502,9.487328,2.027954,0.000000,0.081993,2.035686,10.432002
2018,23.744462,31.567880,6.194785,14.300863,6.317323,4.600696,11.373539,14.062038,9.484561,0.247992,...,2.907483,11.974157,0.250078,10.898616,12.348632,2.245139,0.000000,0.077195,2.562906,13.673837
2019,24.481399,33.361357,6.963326,17.876620,6.432890,4.149221,14.504167,14.467266,9.523887,0.321072,...,3.040304,12.707678,0.256901,11.411698,13.177772,3.973217,0.000000,0.298343,2.948408,16.120056
2020,18.039923,26.559996,5.327996,11.444862,4.819562,2.569009,10.816713,10.432634,7.322671,0.344744,...,2.282703,10.280731,0.131220,9.161791,10.413235,3.509060,0.041586,0.141756,2.432346,11.733066


In [30]:
# Sort the pivot table by TF-IDF averages (mean across years)
pivot_table_sorted = pivot_table.mean(axis=0).sort_values(ascending=False).index

# Reorder the rows in the pivot table
pivot_table = pivot_table.reindex(columns=pivot_table_sorted)
pivot_table

keyword,active listen,goal,decision,accountability,decision make,creativity,analytical,curiosity,communication,detail,...,negotiation,speak,influence,hospitality,kindness,conflict management,social skill,persuasion,interpersonal communication,self organize
year,,,,,,,,,,,,,,,,,,,,,
2012,7.015836,6.581417,5.682216,5.288147,5.245536,4.699208,2.953944,2.772001,3.114192,2.325106,...,0.688736,0.508012,0.125030,0.242065,0.135655,0.000000,0.065533,0.000000,0.000000,0.000000
2013,18.871601,16.362968,14.719390,14.495105,13.324122,11.333971,8.417204,8.043272,7.066911,5.540286,...,1.188691,1.399236,0.838234,0.541640,0.182470,0.093575,0.260520,0.041401,0.000000,0.000000
2014,19.609159,16.623229,14.814527,17.301820,13.493619,11.304135,8.839863,9.494525,8.649410,7.040782,...,2.284339,1.437729,1.058441,0.965366,0.467387,0.340164,0.183462,0.055788,0.066916,0.000000
2015,23.210459,18.863679,16.230256,18.941407,14.806262,13.284320,10.073015,10.121263,10.601702,8.967727,...,1.909728,1.218875,0.869139,1.073519,0.711597,0.146371,0.053724,0.000000,0.083796,0.000000
2016,27.303306,19.654788,16.964889,19.969392,15.769658,13.963329,11.011923,9.868861,10.334202,9.983727,...,1.663222,1.326191,1.334700,1.648460,0.750242,0.140398,0.078597,0.080923,0.194028,0.000000
2017,27.051080,21.969447,21.079074,20.918112,19.781935,15.413237,12.210576,11.757711,12.769102,10.921868,...,1.533590,2.035686,1.060318,2.331166,1.095656,0.235837,0.081993,0.145285,0.000000,0.000000
2018,31.567880,25.630135,25.578495,23.744462,23.965831,19.381872,14.300863,14.701140,14.062038,13.549150,...,2.277427,2.562906,1.920117,1.245871,1.843940,0.247992,0.077195,0.250078,0.066300,0.000000
2019,33.361357,25.504441,28.882955,24.481399,26.486308,22.713580,17.876620,16.443022,14.467266,14.782979,...,2.713902,2.948408,2.555669,1.694510,1.782085,0.321072,0.298343,0.256901,0.159367,0.000000
2020,26.559996,20.674427,19.996127,18.039923,18.496869,16.225867,11.444862,12.015149,10.432634,10.889760,...,2.166060,2.432346,1.851677,1.245104,1.199101,0.344744,0.141756,0.131220,0.053618,0.041586


In [32]:
# Save soft skills TF-IDF
# pivot_table.to_csv(os.path.join(env_info['project_dir'], 'reports', 'timeline_tfidf.csv'))